In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import itertools

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

from lightfm import LightFM
from lightfm.evaluation import precision_at_k, recall_at_k

# Функции из 1-ого вебинара
import os, sys

module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)
    
# from src.metrics import precision_at_k, recall_at_k
from src.utils import prefilter_items

C:\ProgramData\Anaconda3\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


In [2]:
data = pd.read_csv('../data/retail_train.csv')
item_features = pd.read_csv('../data/product.csv')
user_features = pd.read_csv('../data/hh_demographic.csv')

# column processing
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)

# train test split
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


## 1. Filter items

In [3]:
n_items_before = data_train['item_id'].nunique()

data_train = prefilter_items(data_train, 5000, item_features)

n_items_after = data_train['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

Decreased # items from 86865 to 5001


## 2. Prepare data set

In [4]:
# Заведем фиктивный item_id (если юзер покупал товары из топ-5000, то он "купил" такой товар)
# data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробоват ьдругие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(2)

item_id,51716,818980,818981,819063,819112,819227,819255,819304,819308,819330,...,13416351,13417590,13417591,13506200,13511722,13512017,13512060,13654702,13672065,13877192
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
data_test = data_test[data_test['item_id'].isin(data_train['item_id'].unique())]
test_user_item_matrix = pd.pivot_table(data_test, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробовать другие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

test_user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

In [6]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

## 3. Prepare user and item features


In [7]:
user_feat = pd.DataFrame(user_item_matrix.index)
user_feat = user_feat.merge(user_features, on='user_id', how='left')
user_feat.set_index('user_id', inplace=True)

item_feat = pd.DataFrame(user_item_matrix.columns)
item_feat = item_feat.merge(item_features, on='item_id', how='left')
item_feat.set_index('item_id', inplace=True)

user_feat.head(2)

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc
user_id,,,,,,,
1,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
user_feat_lightfm = pd.get_dummies(user_feat, columns=user_feat.columns.tolist())
item_feat_lightfm = pd.get_dummies(item_feat, columns=item_feat.columns.tolist())

In [9]:
model = LightFM(no_components=30,
                loss='bpr', # 'warp'
                learning_rate=0.05, 
                item_alpha=0.1, user_alpha=0.1, 
                random_state=42)

In [10]:
model.fit((sparse_user_item > 0) * 1,  # user-item matrix из 0 и 1
          sample_weight=coo_matrix(user_item_matrix),
          user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
          item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
          epochs=15, 
          num_threads=4)

In [11]:
train_precision = precision_at_k(model, sparse_user_item, 
                                 user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                                 item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                                 k=5).mean()

train_precision

0.221217

# Домашнее задание

## Подбор оптимальных гиперпараметров для LightFM

- Постройте модели с помощью библиотеки LightFM, изменяя следующие параметры
  - функция потерь, регуляризация
  - количество компонент
  - отдельно постройте модели, используя только матрицу взаимодействий и матрицу взаимодействий + признаки (набор признаков может быть различным, например как на вебинаре)
  
- Посчитайте метрики (Precision@5, MAP@5) для разных наборов гиперпараметров и выберете лучший набор


In [12]:
params = {'no_components': [10, 20, 30],
              'loss':          ['bpr', 'warp'],
              'learning_rate': [0.001, 0.1],
              'alpha':         [0.01, 0.1]}


In [13]:
def grid_search(sparse_user_item, params, N=5):
    res = []
    test_data = csr_matrix(test_user_item_matrix).tocsr()
    user_features = csr_matrix(user_feat_lightfm.values).tocsr()
    item_features = csr_matrix(item_feat_lightfm.values).tocsr()
    
    for no_components, loss, learning_rate, alpha in itertools.product(*params.values()):
        model = LightFM(no_components=no_components,
                        loss=loss, # 'warp'
                        learning_rate=learning_rate, 
                        item_alpha=alpha, 
                        user_alpha=alpha, 
                        random_state=42)

        model.fit((sparse_user_item > 0) * 1,  # user-item matrix из 0 и 1
                  sample_weight=coo_matrix(user_item_matrix),
                  user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                  item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                  epochs=15, 
                  num_threads=4)
        
        precision = precision_at_k(model,
                                   test_data,
                                   user_features=user_features,
                                   item_features=item_features,
                                   k=N).mean()
        score = {
            "no_components": no_components,
            "loss": loss,
            "learning_rate": learning_rate,
            "alpha": alpha, 
            "precision": precision
        }
        print(score)
        res.append(score)
    return res




In [14]:
%%time
scores = grid_search(sparse_user_item, params)
df = pd.DataFrame(scores, index=range(len(scores))).sort_values('precision', ascending=False)
best_params = df.head(1).to_dict()
best_params

{'no_components': 10, 'loss': 'bpr', 'learning_rate': 0.001, 'alpha': 0.01, 'precision': 0.413691}
{'no_components': 10, 'loss': 'bpr', 'learning_rate': 0.001, 'alpha': 0.1, 'precision': 0.32738194}
{'no_components': 10, 'loss': 'bpr', 'learning_rate': 0.1, 'alpha': 0.01, 'precision': 0.32754207}
{'no_components': 10, 'loss': 'bpr', 'learning_rate': 0.1, 'alpha': 0.1, 'precision': 0.2163331}
{'no_components': 10, 'loss': 'warp', 'learning_rate': 0.001, 'alpha': 0.01, 'precision': 0.26821458}
{'no_components': 10, 'loss': 'warp', 'learning_rate': 0.001, 'alpha': 0.1, 'precision': 0.17718174}
{'no_components': 10, 'loss': 'warp', 'learning_rate': 0.1, 'alpha': 0.01, 'precision': 0.34051242}
{'no_components': 10, 'loss': 'warp', 'learning_rate': 0.1, 'alpha': 0.1, 'precision': 0.5895917}
{'no_components': 20, 'loss': 'bpr', 'learning_rate': 0.001, 'alpha': 0.01, 'precision': 0.32754207}
{'no_components': 20, 'loss': 'bpr', 'learning_rate': 0.001, 'alpha': 0.1, 'precision': 0.33787033}
{'n

{'no_components': {7: 10},
 'loss': {7: 'warp'},
 'learning_rate': {7: 0.1},
 'alpha': {7: 0.1},
 'precision': {7: 0.5895916819572449}}